<a href="https://colab.research.google.com/github/harshith7823/CS760-Project/blob/clean_clinical_data/clean_clinical_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [2]:
def fill_death_col(clinical_data):
  
  male_obese = (clinical_data['Sex'] == 1) & (clinical_data['BMI'] > 30)
  female_obese = (clinical_data['Sex'] == -1) & (clinical_data['BMI'] > 30)
  male_underweight = (clinical_data['Sex'] == 1) & (clinical_data['BMI'] < 18.5)
  female_underweight = (clinical_data['Sex'] == -1) & (clinical_data['BMI'] < 18.5)
  clinical_data['BMI_reduction_factor'] = 0
  clinical_data.loc[male_obese, 'BMI_reduction_factor'] = 4.2
  clinical_data.loc[female_obese, 'BMI_reduction_factor'] = 3.5
  clinical_data.loc[male_underweight, 'BMI_reduction_factor'] = 4.3
  clinical_data.loc[female_underweight, 'BMI_reduction_factor'] = 4.5

  clinical_data['Tobacco_reduction_factor'] = np.where(clinical_data['Tobacco']==1,10,0)

  #Leave na on rows with no death data, fill the rest with age*365+death days 
  clinical_data['Age_at_death'] = np.where(clinical_data['DEATH [d from CT]'].isna(), np.nan, clinical_data['Age at CT']*365 + clinical_data['DEATH [d from CT]'])
  
  # fill death with avg lifespan - alpha(reduction for tobacco) - beta(reduction for bmi)
  corr_bmi_death = clinical_data['BMI'].corr(clinical_data['Age_at_death'])
  corr_tobacco_death = clinical_data['Tobacco'].corr(clinical_data['Age_at_death'])

  print(corr_bmi_death)
  print(corr_tobacco_death)

  #normalize
  alpha = corr_bmi_death/(corr_bmi_death+corr_tobacco_death)
  beta = corr_tobacco_death/(corr_bmi_death+corr_tobacco_death)

  print(alpha)
  print(beta)
  
  avg_lifespan_male=76.3  
  avg_lifespan_female=81.2

  clinical_data['avg_life'] = np.where(clinical_data['Sex']==1, avg_lifespan_male, avg_lifespan_female)
  clinical_data['avg_life'] = clinical_data['avg_life'] - alpha*(clinical_data['BMI_reduction_factor'])-beta*(clinical_data['Tobacco_reduction_factor'])
  clinical_data['DEATH [d from CT]'].fillna((clinical_data['avg_life']-clinical_data['Age at CT'])*365, inplace=True)
  

In [3]:
def preprocess_clinical_data(oppScrData, mean=True):
    clinical_data = oppScrData.filter(['BMI','BMI >30', 'Sex', 'Tobacco', 'Met Sx', 'FRAX 10y Fx Prob (Orange-w/ DXA)',
                                'FRAX 10y Hip Fx Prob (Orange-w/ DXA)','FRS 10-year risk (%)', 'DEATH [d from CT]' , 'Age at CT'], axis=1)
    # Replace all _,X,blanks with nan
    clinical_data = clinical_data.replace(r'_', np.nan, regex=True)
    clinical_data = clinical_data.replace(r'X', np.nan, regex=True)
    clinical_data = clinical_data.replace(r'^\s*$', np.nan, regex=True)

    # Fill na in bmi column with mean
    clinical_data['BMI'].fillna(value=clinical_data['BMI'].mean(skipna=True), inplace=True)
    
    # Fill na in bmi>30 column based on bmi col
    clinical_data.loc[clinical_data.BMI>30, 'BMI >30'] = 1
    clinical_data.loc[clinical_data.BMI<=30, 'BMI >30'] = -1
    
    clinical_data['Sex'] = np.where(clinical_data['Sex']=='Male',1,-1)
    clinical_data['Met Sx'] = np.where(clinical_data['Met Sx']=='Y',1,-1) 

    # Treat no data in tobacco as no tobacco usage 
    clinical_data['Tobacco'] = np.where(clinical_data['Tobacco']=='Yes',1,-1) 
    
    #Fill death col
    fill_death_col(clinical_data)

    clinical_data['FRS 10-year risk (%)'] = clinical_data['FRS 10-year risk (%)'].replace("<1", 0.01, regex=True)
    clinical_data['FRS 10-year risk (%)'] = clinical_data['FRS 10-year risk (%)'].replace(">30", 0.30, regex=True)
    clinical_data['FRS 10-year risk (%)'] =  clinical_data['FRS 10-year risk (%)'] * 100
 
    cols_to_be_filled = ['FRAX 10y Fx Prob (Orange-w/ DXA)','FRAX 10y Hip Fx Prob (Orange-w/ DXA)','FRS 10-year risk (%)']
    for c in cols_to_be_filled:
      if mean:  
        clinical_data[c].fillna(value=clinical_data[c].mean(skipna=True), inplace=True)
      else :
        clinical_data[c].fillna(value=clinical_data[c].median(skipna=True), inplace=True)

    return [clinical_data, np.array(clinical_data, dtype=np.float32)]

In [4]:
def normalize_data(data):
    n = data.shape[1]    
    for i in range(n-1):
      data[:,i] = (data[:,i] - np.min(data[:,i]))/(np.max(data[:,i])- np.min(data[:,i]))
    return data

In [7]:
oppScrData = pd.read_excel (r'OppScrData.xlsx')  

In [8]:
clinical_data, clinical_data_np = preprocess_clinical_data(oppScrData)
clinical_data_np = normalize_data(clinical_data_np)

-0.1361782164594117
-0.018215285783995318
0.882020386095794
0.11797961390420597


In [74]:

clinical_data

,BMI,BMI >30,Sex,Tobacco,Met Sx,FRAX 10y Fx Prob (Orange-w/ DXA),FRAX 10y Hip Fx Prob (Orange-w/ DXA),FRS 10-year risk (%),DEATH [d from CT],Age at CT,BMI_reduction_factor,Tobacco_reduction_factor,Age_at_death,avg_life
0,37.7,1,1,-1,-1,4.60,1.04,3.807755,967.000000,73,4.2,0,27612.0,72.595514
1,30.1,1,-1,-1,-1,5.01,0.27,3.807755,359.000000,55,3.5,0,20434.0,78.112929
2,28.1,-1,-1,-1,-1,4.13,0.21,3.807755,10658.000000,52,0.0,0,NaN,81.200000
3,25.0,-1,1,1,-1,5.46,0.75,6.000000,2351.000000,60,0.0,10,24251.0,75.120204
4,22.4,-1,1,1,-1,11.65,8.40,16.000000,658.000000,88,0.0,10,32778.0,75.120204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9217,28.3,-1,1,-1,-1,4.97,0.27,5.000000,8869.500000,52,0.0,0,NaN,76.300000
9218,23.4,-1,1,1,-1,3.30,0.25,6.000000,8803.874409,51,0.0,10,NaN,75.120204
9219,43.6,1,-1,-1,1,8.26,0.34,2.000000,5516.218957,63,3.5,0,NaN,78.112929
9220,26.9,-1,-1,-1,-1,5.45,0.32,1.000000,9563.000000,55,0.0,0,NaN,81.200000


In [73]:
clinical_data_np

array([[ 0.42022473,  1.        ,  1.        , ...,  0.        ,
                nan, 72.59551   ],
       [ 0.33483148,  1.        ,  0.        , ...,  0.        ,
                nan, 78.11293   ],
       [ 0.31235957,  0.        ,  0.        , ...,  0.        ,
                nan, 81.2       ],
       ...,
       [ 0.48651683,  1.        ,  0.        , ...,  0.        ,
                nan, 78.11293   ],
       [ 0.2988764 ,  0.        ,  0.        , ...,  0.        ,
                nan, 81.2       ],
       [ 0.21910113,  0.        ,  1.        , ...,  0.        ,
                nan, 76.3       ]], dtype=float32)

In [51]:
from sklearn.svm import SVR

from sklearn.model_selection import GridSearchCV

In [45]:
X=clinical_data.iloc[:, 0:8]
X
y=clinical_data.iloc[:, 8]
y


0         967.000000
1         359.000000
2       10658.000000
3        2351.000000
4         658.000000
            ...     
9217     8869.500000
9218     8803.874409
9219     5516.218957
9220     9563.000000
9221     5584.500000
Name: DEATH [d from CT], Length: 9222, dtype: float64

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=0)

In [53]:
parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.5, 10],'gamma': [1e-7, 1e-4],'epsilon':[0.1,0.2,0.5,0.3]}
svr = SVR()
clf = GridSearchCV(svr, parameters, refit = True, verbose = 3)
clf.fit(X_train,y_train)
clf.best_params_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END C=1.5, epsilon=0.1, gamma=1e-07, kernel=linear;, score=0.581 total time=   1.4s
[CV 2/5] END C=1.5, epsilon=0.1, gamma=1e-07, kernel=linear;, score=0.631 total time=   1.4s
[CV 3/5] END C=1.5, epsilon=0.1, gamma=1e-07, kernel=linear;, score=0.591 total time=   1.4s
[CV 4/5] END C=1.5, epsilon=0.1, gamma=1e-07, kernel=linear;, score=0.612 total time=   1.4s
[CV 5/5] END C=1.5, epsilon=0.1, gamma=1e-07, kernel=linear;, score=0.606 total time=   1.4s
[CV 1/5] END C=1.5, epsilon=0.1, gamma=1e-07, kernel=rbf;, score=-0.025 total time=   3.2s
[CV 2/5] END C=1.5, epsilon=0.1, gamma=1e-07, kernel=rbf;, score=-0.015 total time=   3.2s
[CV 3/5] END C=1.5, epsilon=0.1, gamma=1e-07, kernel=rbf;, score=-0.030 total time=   3.1s
[CV 4/5] END C=1.5, epsilon=0.1, gamma=1e-07, kernel=rbf;, score=-0.024 total time=   3.0s
[CV 5/5] END C=1.5, epsilon=0.1, gamma=1e-07, kernel=rbf;, score=-0.018 total time=   3.0s
[CV 1/5] END C=1.5

[CV 4/5] END C=1.5, epsilon=0.5, gamma=0.0001, kernel=poly;, score=-0.024 total time=   1.7s
[CV 5/5] END C=1.5, epsilon=0.5, gamma=0.0001, kernel=poly;, score=-0.018 total time=   1.7s
[CV 1/5] END C=1.5, epsilon=0.3, gamma=1e-07, kernel=linear;, score=0.581 total time=   1.4s
[CV 2/5] END C=1.5, epsilon=0.3, gamma=1e-07, kernel=linear;, score=0.631 total time=   1.4s
[CV 3/5] END C=1.5, epsilon=0.3, gamma=1e-07, kernel=linear;, score=0.591 total time=   1.4s
[CV 4/5] END C=1.5, epsilon=0.3, gamma=1e-07, kernel=linear;, score=0.612 total time=   1.4s
[CV 5/5] END C=1.5, epsilon=0.3, gamma=1e-07, kernel=linear;, score=0.606 total time=   1.4s
[CV 1/5] END C=1.5, epsilon=0.3, gamma=1e-07, kernel=rbf;, score=-0.025 total time=   3.0s
[CV 2/5] END C=1.5, epsilon=0.3, gamma=1e-07, kernel=rbf;, score=-0.015 total time=   3.0s
[CV 3/5] END C=1.5, epsilon=0.3, gamma=1e-07, kernel=rbf;, score=-0.030 total time=   3.0s
[CV 4/5] END C=1.5, epsilon=0.3, gamma=1e-07, kernel=rbf;, score=-0.024 tota

[CV 4/5] END C=10, epsilon=0.2, gamma=0.0001, kernel=poly;, score=-0.023 total time=   1.6s
[CV 5/5] END C=10, epsilon=0.2, gamma=0.0001, kernel=poly;, score=-0.016 total time=   1.6s
[CV 1/5] END C=10, epsilon=0.5, gamma=1e-07, kernel=linear;, score=0.584 total time=   1.5s
[CV 2/5] END C=10, epsilon=0.5, gamma=1e-07, kernel=linear;, score=0.623 total time=   1.5s
[CV 3/5] END C=10, epsilon=0.5, gamma=1e-07, kernel=linear;, score=0.580 total time=   1.5s
[CV 4/5] END C=10, epsilon=0.5, gamma=1e-07, kernel=linear;, score=0.605 total time=   1.5s
[CV 5/5] END C=10, epsilon=0.5, gamma=1e-07, kernel=linear;, score=0.593 total time=   1.5s
[CV 1/5] END C=10, epsilon=0.5, gamma=1e-07, kernel=rbf;, score=-0.025 total time=   3.0s
[CV 2/5] END C=10, epsilon=0.5, gamma=1e-07, kernel=rbf;, score=-0.015 total time=   3.0s
[CV 3/5] END C=10, epsilon=0.5, gamma=1e-07, kernel=rbf;, score=-0.030 total time=   3.0s
[CV 4/5] END C=10, epsilon=0.5, gamma=1e-07, kernel=rbf;, score=-0.024 total time=   3

{'C': 1.5, 'epsilon': 0.2, 'gamma': 1e-07, 'kernel': 'linear'}

In [55]:
p=clf.predict(X_test)

In [58]:
p

array([ 6196.61087063,  8915.38043516,  7752.60171168, ...,
        8517.90770703, 10712.77516395,  6044.09437446])

In [63]:
from sklearn.neighbors import KNeighborsRegressor

In [76]:
k_range = list(range(2,31))
weight_options = ["uniform", "distance"]

param_grid = dict(n_neighbors = k_range, weights = weight_options)

In [77]:
knn = KNeighborsRegressor()

grid = GridSearchCV(knn, param_grid, cv = 10, scoring = 'accuracy')
grid.fit(X_train,y_train)

/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/Users/sanyamkakkar/op

/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/Users/sanyamkakkar/op

/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/Users/sanyamkakkar/op

/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/Users/sanyamkakkar/op

/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/Users/sanyamkakkar/op

...

/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/Users/sanyamkakkar/op

/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/Users/sanyamkakkar/op

/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/Users/sanyamkakkar/op

/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/sanyamkakkar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/Users/sanyamkakkar/op

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                         14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                                         24, 25, 26, 27, 28, 29, 30],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [78]:
p2=grid.predict(X_test)

In [79]:
p2

array([ 6666.93720462,  8219.87440925,  7489.87440925, ...,
        9198.        , 11023.        ,  4971.37440925])

In [80]:
grid.best_params_

{'n_neighbors': 2, 'weights': 'uniform'}